In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_curve, auc
from scipy.stats import ks_2samp

##### Carregando o dataset

In [2]:
df_train = pd.read_csv('../data/treated_data/train.csv')
df_val = pd.read_csv('../data/treated_data/validation.csv')
df_test = pd.read_csv('../data/treated_data/test.csv')

In [3]:
X_train = df_train.drop(['satisfaction'], axis=1)
y_train = df_train['satisfaction'].values

X_val = df_val.drop(['satisfaction'], axis=1)
y_val = df_val['satisfaction'].values

X_test = df_test.drop(['satisfaction'], axis=1)
y_test = df_test['satisfaction'].values

##### Ajustando variáveis one-hot-encoding para string

In [ ]:
columns_to_concat = [
    'Age',
    'Flight Distance',
    'Departure Delay in Minutes',
    'Arrival Delay in Minutes',
    'Gender',
    'Customer Type',
    'Type of Travel',
    'Class'
]

In [ ]:
def one_hot_to_string(df: pd.DataFrame, col: list) -> pd.DataFrame:
    for prefix in col:
        cols_to_concat = [c for c in df.columns if c.startswith(prefix + '_')]
        
        if cols_to_concat:
            df[prefix] = df[cols_to_concat].astype(str).agg(''.join, axis=1)
            
            df.drop(cols_to_concat, axis=1, inplace=True)
            
    return df

In [ ]:
X_train = one_hot_to_string(df=X_train, col=columns_to_concat)
X_val = one_hot_to_string(df=X_val, col=columns_to_concat)
X_test = one_hot_to_string(df=X_test, col=columns_to_concat)

In [ ]:
X_train.columns

In [ ]:
X_train = X_train.values
X_val = X_val.values
X_test = X_test.values

##### Criando o modelo da Random Forest

In [ ]:
model = RandomForestClassifier(
    n_estimators=10,
    max_depth=10,
    criterion='gini',
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=df_train.shape[1]
)

In [ ]:
model.fit(X=X_train, y=y_train)

##### Avaliando o modelo

In [ ]:
def plot_confusion_matrix(y_pred: np.array, y_test: np.array) -> None:
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Unsatisfied', 'Satisfied'], yticklabels=['Unsatisfied', 'Satisfied'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
y_pred_val = model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print(f'Accuracy no conjunto de Validação: {accuracy_val:.4f}')

In [ ]:
plot_confusion_matrix(y_pred=y_pred_val, y_test=y_val)

In [ ]:
def plot_roc_curve(y_pred: np.array, y_test: np.array) -> None:

    fpr, tpr, thresholds = roc_curve(y_pred, y_test)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
plot_roc_curve(y_pred_val, y_val)

In [ ]:
y_pred_test = model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f'Accuracy no conjunto de Teste: {accuracy_test:.4f}')

In [ ]:
plot_confusion_matrix(y_pred=y_pred_test, y_test=y_test)

In [ ]:
plot_roc_curve(y_pred_test, y_test)

#Teste K-S

1. Estatística K-S (statistic)
- Valor da estatística: Este valor é a máxima diferença absoluta entre as funções de distribuição empírica acumulada (ECDFs) das duas amostras comparadas. Quanto menor for a estatística, mais próximas são as distribuições das duas amostras.
- Interpretação ideal: Um valor baixo da estatística K-S (próximo de 0) sugere que as distribuições das probabilidades previstas pelo modelo são consistentes entre os subconjuntos de dados (como treino e teste). Isso implica que o modelo é estável e generaliza bem para novos dados, assumindo que não há sobreajuste.
2. P-valor
- Valor do p-valor: Este valor indica a probabilidade de observar uma diferença tão extrema quanto a estatística K-S, assumindo que as duas amostras são da mesma distribuição.
Interpretação ideal:
- P-valor alto (por exemplo, > 0.05): Não há evidência estatística suficiente para rejeitar a hipótese de que as distribuições das amostras são iguais. Isto é, parece que o modelo se comporta de forma consistente entre os grupos comparados.
- P-valor baixo (por exemplo, < 0.05): Há evidência estatística suficiente para rejeitar a hipótese de que as amostras vêm da mesma distribuição. Isso pode indicar que o modelo está comportando de forma diferente entre os subconjuntos de dados, o que pode ser um sinal de sobreajuste ou de problemas de generalização.

In [ ]:
prob_train = model.predict_proba(X_train)  # Probabilidades da classe positiva para o conjunto de treino
prob_test = model.predict_proba(X_test) # Probabilidades da classe positiva para o conjunto de teste

len(X_train)

In [ ]:
prob_train = model.predict_proba(X_train)[:, 1]  # Probabilidades da classe positiva para o conjunto de treino
prob_test = model.predict_proba(X_test)[:, 1]  # Probabilidades da classe positiva para o conjunto de teste

statistic, p_value = ks_2samp(prob_train, prob_test)

print("K-S statistic:", statistic)
print("P-value:", p_value)